In [ ]:
# true arma aut 

In [ ]:
function (phi = 0, theta = 0, lag.max = 25, vara = 1, plot = TRUE) 
{
    p = length(phi)
    q = length(theta)
    if (all(phi == 0)) {
        ar = NA
        p = 0
    }
    if (all(theta == 0)) {
        ma = NA
        q = 0
    }
    ip = max(p, q + 1)
    ipm1 = ip - 1
    ipp1 = ip + 1
    lag.maxp1 = lag.max + 1
    g = rep(0, lag.maxp1)
    d = rep(0, lag.maxp1)
    aut1 = rep(0, lag.maxp1)
    spec = rep(0, 251)
    a = matrix(rep(0, 676), ncol = 26)
    papp = ip - p
    PHI <- phi
    while (papp > 0) {
        zp <- rep(0, papp)
        PHI <- append(phi, zp)
        papp = -1
    }
    PHI
    qapp = ip - q
    qapp
    THETA <- theta
    while (qapp > 0) {
        zq = rep(0, qapp)
        THETA = append(theta, zq)
        qapp = -1
    }
    THETA
    if (ip <= 1) {
        d1 = (PHI[1] - THETA[1]) * vara
        dn = 1 - PHI[1]^2
        g[1] = (vara - THETA[1] * d1 - THETA[1] * vara * PHI[1])/dn
    }
    if (ip > 1) {
        ipm1 = ip - 1
        ipp1 = ip + 1
        d[1] = vara
        for (i in 2:ip) {
            im1 = i - 1
            for (j in 1:im1) {
                d[i] = d[i] + PHI[j] * d[i - j]
            }
            d[i] = d[i] - THETA[i - 1] * vara
        }
        for (i in 1:ipm1) {
            g[i] = 0
            k = ip - i
            for (j in 1:k) {
                g[i] = g[i] - THETA[j + i - 1] * d[j + 1]
            }
        }
        g[ip] = 0
        g[1] = g[1] + d[1]
        for (i in 2:ip) {
            g[i] = g[i] - THETA[i - 1] * d[1]
        }
        for (i in 1:ip) {
            a[i, i] = 1
        }
        for (i in 1:ipm1) {
            for (j in 1:ip) {
                ii = abs(i - j + 1) + 1
                a[j, ii] = a[j, ii] - PHI[i]
            }
        }
        for (i in 2:ip) {
            ii = abs(ip - i + 1) + 1
            a[i, ii] = a[i, ii] - PHI[ip]
        }
        for (i in 1:ip) {
            ii = ip - i + 1
            a[1, i] = a[1, i] - PHI[ip] * PHI[ii]
        }
        for (k in 2:ip) {
            m = k - 1
            for (i in k:ip) {
                for (j in k:ip) {
                  a[i, j] = a[i, j] - a[i, m] * a[m, j]/a[m, 
                    m]
                }
                g[i] = g[i] - a[i, m] * g[m]/a[m, m]
            }
        }
        g[ip] = g[ip]/a[ip, ip]
        for (i in 1:ipm1) {
            m = ip - i
            ctc = 0
            for (j in 1:i) {
                k = ip - j + 1
                ctc = ctc + a[m, k] * g[k]
            }
            g[m] = (g[m] - ctc)/a[m, m]
        }
    }
    gvar = g[1]
    lag.maxp1 = lag.max + 1
    for (i in 1:lag.maxp1) {
        aut1[i] = g[i]/gvar
    }
    if (p > 0) {
        for (k in ipp1:lag.maxp1) {
            g[k] = 0
            for (j in 1:p) {
                g[k] = g[k] + PHI[j] * g[k - j]
            }
        }
    }
    gvar = g[1]
    for (i in 1:lag.maxp1) {
        aut1[i] = g[i]/gvar
    }
    if (plot == "TRUE") {
        k = 0:lag.max
        cex.labs <- c(0.9, 0.8, 0.9)
        numrows <- 1
        numcols <- 1
        par(mfrow = c(numrows, numcols), mar = c(3.8, 2.5, 1, 
            1))
        plot(k, aut1, type = "h", xaxt = "n", yaxt = "n", cex = 0.4, 
            cex.lab = 0.75, cex.axis = 0.75, lwd = 0.75, xlab = "", 
            ylab = "", ylim = c(-1, 1))
        abline(h = 0)
        axis(side = 1, cex.axis = 0.8, mgp = c(3, 0.15, 0), tcl = -0.3)
        axis(side = 2, las = 1, cex.axis = 0.8, mgp = c(3, 0.4, 
            0), tcl = -0.3)
        mtext(side = c(1, 2, 1), cex = cex.labs, text = c("Lag", 
            "", "True Autocorrelations"), line = c(1, 1.1, 2.1))
    }
    out1 = list(acf = aut1, acv = g)
    return(out1)
}